## Import Modules

In [1]:
# Set paths
import os
from imp import reload

# Data manipulation
import pandas as pd
import numpy as np
from scipy import stats

# Date manipulation
import datetime as dt
import calendar
calendar.setfirstweekday(calendar.SUNDAY) 

# Custom package for data preprocessing
import mytools as mt 

# Set notebook options
pd.set_option("display.max_columns", 150)
pd.set_option("display.max_rows", 10000)
pd.set_option("display.float_format", lambda x: '%.2f' % x)

# Pretty display of multiple functions in a cell
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

### File Location

In [2]:
# Data paths
import filepaths

## Load Data

In [3]:
df_master = pd.read_csv(filepaths.master_file_data, sep=',')

df_transactions = pd.read_csv(filepaths.interim_transactions_data, sep=',')

In [4]:
df_transactions.head(2)

,trans_id,customer_id,trans_date,prod_subcat_code,prod_cat_code,qty,rate,cogs,total_amt,store_type,returned_item,drop_record,prod_cat,prod_subcat,counter,assessment_date,duration,trans_date.day,trans_date.month_num,trans_date.year,trans_date.year_month,trans_date.hour,trans_date.weekday,trans_date.week_of_year,trans_date.month,trans_date.weekday_num,trans_date.week_of_month,first_purchase_date,first_purchase.cohort,purchase_date.cohort,customer_type.period,returned_item_before
0,25890929042,266783,2011-09-23,1,2,4,1321,554.82,5838.82,e-Shop,yes,yes,Footwear,Mens,1,2014-12-02,1.00,23,9,2011,2011_09,0,06_Friday,38,09_Sep,6,month.week4,2011-09-23,2011_09,2011_09,new,yes
1,25890929042,266783,2011-09-24,1,2,-4,-1321,554.82,-5838.82,e-Shop,yes,no,Footwear,Mens,1,2014-12-02,393.00,24,9,2011,2011_09,0,07_Saturday,38,09_Sep,7,month.week4,2011-09-23,2011_09,2011_09,existing,yes


In [5]:
df_transactions = df_transactions[~((df_transactions['returned_item'] == 'yes') & (df_transactions['drop_record'] == 'yes'))]

mt.check_unique_no(df_transactions, ['customer_id'])
df_transactions.shape
df_transactions.head(2)

Data has 5506 unique customer_id


(20996, 32)

,trans_id,customer_id,trans_date,prod_subcat_code,prod_cat_code,qty,rate,cogs,total_amt,store_type,returned_item,drop_record,prod_cat,prod_subcat,counter,assessment_date,duration,trans_date.day,trans_date.month_num,trans_date.year,trans_date.year_month,trans_date.hour,trans_date.weekday,trans_date.week_of_year,trans_date.month,trans_date.weekday_num,trans_date.week_of_month,first_purchase_date,first_purchase.cohort,purchase_date.cohort,customer_type.period,returned_item_before
1,25890929042,266783,2011-09-24,1,2,-4,-1321,554.82,-5838.82,e-Shop,yes,no,Footwear,Mens,1,2014-12-02,393.00,24,9,2011,2011_09,0,07_Saturday,38,09_Sep,7,month.week4,2011-09-23,2011_09,2011_09,existing,yes
2,98477711300,266783,2012-10-21,4,1,3,93,29.30,308.30,TeleShop,no,yes,Clothing,Mens,1,2014-12-02,122.00,21,10,2012,2012_10,0,01_Sunday,42,10_Oct,1,month.week3,2011-09-23,2011_09,2012_10,existing,no


In [6]:
df_transactions['trans_date'] = pd.to_datetime(df_transactions['trans_date'])

df_transactions['trans_date'].sort_values().min()
df_transactions['trans_date'].sort_values().max()

Timestamp('2011-01-02 00:00:00')

Timestamp('2014-12-02 00:00:00')

### RFM Customer Segmentation

In [7]:
EXTRACTION_DATE = dt.datetime(2014,12,2)

In [8]:
def categorize_customers(rfm_score):
    x = rfm_score
    if (x == '111'):
        return 'best customers'
    elif (x == '311'):
        return 'almost lost'
    elif (x == '411'):
        return 'lost customers'
    elif (x == '444'):
        return 'lost cheap customers'
    else:
        return 'other'  

In [9]:
df_rfm= df_transactions.groupby('customer_id', as_index=False).agg({'trans_date': lambda date: (EXTRACTION_DATE - date.max()).days,
                                    'trans_id': lambda num: len(num),
                                    'total_amt': lambda price: price.sum()})

In [10]:
df_rfm.rename(columns={'trans_date':'recency', 'trans_id':'frequency', 'total_amt':'monetary'}, inplace=True)

In [11]:
df_rfm['r_quartile'] = pd.qcut(df_rfm['recency'], 4, ['1','2','3','4'])
df_rfm['f_quartile'] = pd.qcut(df_rfm['frequency'], 4, ['4','3','2','1'])
df_rfm['m_quartile'] = pd.qcut(df_rfm['monetary'], 4, ['4','3','2','1'])

In [12]:
df_rfm['RFM_Score'] = df_rfm.r_quartile.astype(str)+ df_rfm.f_quartile.astype(str) + df_rfm.m_quartile.astype(str)

mt.check_unique_no(df_rfm, ['customer_id'])
df_rfm.shape

Data has 5506 unique customer_id


(5506, 8)

In [13]:
mt.check_unique_no(df_master, ['customer_id'])
df_master.shape

Data has 5647 unique customer_id


(5647, 153)

In [14]:
df_master = pd.merge(df_master, df_rfm, on='customer_id', how='left')

In [15]:
df_master['rfm_customer_segment'] = df_master['RFM_Score'].apply(categorize_customers)    

In [16]:
df_master['rfm_customer_segment'] = np.where(((df_master['rfm_customer_segment'] == 'other') & 
                                                (df_master['m_quartile'] == '1')), 'big spender',
                                           np.where(((df_master['rfm_customer_segment'] == 'other') & 
                                                     (df_master['f_quartile'] == '1')), 'loyal customers',
                                                   df_master['rfm_customer_segment']))

df_master[['customer_id', 'rfm_customer_segment']].head()

,customer_id,rfm_customer_segment
0,268408,best customers
1,269696,lost cheap customers
2,268159,almost lost
3,270181,loyal customers
4,268073,other


In [17]:
df_master['rfm_customer_segment'].value_counts(dropna=False)

other                   3512
big spender              954
lost cheap customers     395
loyal customers          363
best customers           271
almost lost              119
lost customers            33
Name: rfm_customer_segment, dtype: int64

In [18]:
mt.check_unique_no(df_master, ['customer_id'])
df_master.shape

mt.check_unique_no(df_transactions, ['customer_id'])
df_transactions.shape

Data has 5647 unique customer_id


(5647, 161)

Data has 5506 unique customer_id


(20996, 32)

## Export Data

In [19]:
df_master.to_csv(filepaths.master_file_data, index=False)
df_transactions.to_csv(filepaths.interim_transactions_data, index=False)